In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier


In [10]:
x_train = pd.read_csv('x_train.csv')
y_train = pd.read_csv('y_train.csv')

Etude de la data (Dimitri)

In [11]:
import seaborn as sns 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor

# Categorical columns in the training data
object_cols = ['station']

print(object_cols)
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
x_train[object_cols] = ordinal_encoder.fit_transform(x_train[object_cols])      



['station']


In [12]:

y_train = y_train.drop('index',axis=1)      


Test de XGBOOST algo de descente de gradient.

In [13]:
#Nombre couple date x gare
# Utiliser groupby pour regrouper les données par gare et date, puis compter le nombre de lignes dans chaque groupe
nombre_total_stations = x_train['station'].nunique()
nombre_date = x_train['date'].nunique()
nombre_couples = nombre_date*nombre_total_stations

# Afficher le nombre de couples gare x date
print("Nombre de couples gare x date :", nombre_couples)

Nombre de couples gare x date : 1282758


In [14]:
from sklearn.metrics import make_scorer
epsilon = 0.00001
# Définir votre fonction de métrique personnalisée
def mape(y_true, y_pred):
    # Implémentez votre calcul de métrique personnalisée ici
    # Par exemple, ici nous calculons la moyenne absolue des différences entre les valeurs vraies et prédites
    score = 0
    for i in range(len(y_true)):
            
        if y_true[i]:
            score += (abs(y_true[i] - y_pred[i])/y_true[i])
        else:
            score += (abs(y_true[i] - y_pred[i]))
    return score/nombre_couples

# Créer un scorer à partir de votre métrique personnalisée
custom_scorer = make_scorer(mape, greater_is_better=False)  # Indiquez True si une valeur plus élevée est meilleure

In [15]:
#test jours année mois
# Convertir la colonne 'date' en datetime

print(x_train.columns)


Index(['date', 'station', 'job', 'ferie', 'vacances'], dtype='object')


In [16]:
# Parcourir les stations uniques et entraîner un modèle pour chacune
# Fusionner les données
import random
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

param_grid_model1 = {
   
    'n_estimators': [10,15,25,30,50,75],
    'max_depth': [3,5, 7,9,11],
    
    # Ajoutez d'autres hyperparamètres à ajuster ici
}


ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
x_train = pd.read_csv('x_train.csv')

x_train['date'] = pd.to_datetime(x_train['date'])

# Créer les colonnes pour le jour, le mois et l'année
x_train['jour'] = x_train['date'].dt.day.astype(int)
x_train['mois'] = x_train['date'].dt.month.astype(int)
x_train['année'] = x_train['date'].dt.year.astype(int)
x_train.drop(['date'],axis = 1)

x_train[object_cols] = ordinal_encoder.fit_transform(x_train[object_cols])
     
y_train = pd.read_csv('y_train.csv')
x_train['y'] = y_train['y']


# Choix aléatoire de 15 stations parmi X_training['station'].unique()
random_stations = random.sample(list(x_train['station'].unique()), 7)
# Créer un dictionnaire pour stocker les modèles par station
model_opti = {}
models1 = {}
models2 = {}
x_test_final = pd.DataFrame(columns =['jour','mois','job', 'ferie', 'vacances','année'])
y_test_final = pd.DataFrame(columns=[0])
for station in x_train['station'].unique():
    station_data = x_train[x_train['station'] == station]
    
    X_station = station_data[['job', 'ferie', 'vacances','station','jour','mois','année']]
    y_station = station_data['y']
    
    X_train, X_test, y_train, y_test = train_test_split(X_station, y_station, test_size=0.2, random_state=42)
    x_test_final = pd.concat([x_test_final,X_test],ignore_index=True)
    y_test_final = pd.concat([y_test_final,y_test],ignore_index=True)
        
    # Créer et entraîner le modèle XGBoost pour cette station
    #model1 = RandomForestRegressor(random_state=42,criterion='friedman_mse')
    if station in random_stations:
        model1 = GridSearchCV(XGBRegressor(random_state = 42),
                                          param_grid_model1, cv=5, verbose=1)
        model1.fit(X_train,y_train)
        best_model1 = model1.best_estimator_
        models1[station] = best_model1
        model_opti[station] = best_model1
    else:
        model1 = XGBRegressor(n_estimators = 25, max_depth = 7,random_state = 42,criterion = custom_scorer)
        model1.fit(X_train,y_train)
        models1[station] = model1
  
    #model2.fit(X_train, y_train)
    
    # Stocker le modèle dans le dictionnaire
   
    #models2[station] = model2
#Faire des prédictions pour chaque station et combiner les résultats

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits


In [17]:
for station, best_model1 in model_opti.items():
    print(f"Meilleurs paramètres pour la station {station} - Modèle 1 (Random Forest):")
    print(best_model1.get_params())
    print()


Meilleurs paramètres pour la station 189.0 - Modèle 1 (Random Forest):
{'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 7, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 25, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}

Meilleurs paramètres pour la stati

In [18]:
y_test_final[0] = y_test_final[0].astype(int)
# y_pred2 = y_test_final.copy()

x_test_final['jour'] = x_test_final['jour'].astype(int)
x_test_final['mois'] = x_test_final['mois'].astype(int)
x_test_final['année'] = x_test_final['année'].astype(int)

x_test_final['job'] = x_test_final['job'].astype(int)
x_test_final['ferie'] = x_test_final['ferie'].astype(int)
x_test_final['vacances'] = x_test_final['vacances'].astype(int)

# for station, model in models2.items():
#     station_data = x_test_final[x_test_final['station'] == station]
#     X_station = station_data[['date','job', 'ferie', 'vacances','station']]
#     predictions = (model.predict(X_station))
#     i=0
#     for index in X_station.index:
#         # Remplacer la valeur prédite dans y_pred à l'index correspondant
#         y_pred2.at[index, 0] = predictions[i]
#         i += 1

y_pred1 = y_test_final.copy()
for station, model in models1.items():
    station_data = x_test_final[x_test_final['station'] == station]
    X_station = station_data[['job', 'ferie', 'vacances','station','jour','mois','année']]
    predictions = (model.predict(X_station))
    i=0
    for index in X_station.index:
        # Remplacer la valeur prédite dans y_pred à l'index correspondant
        y_pred1.at[index, 0] = predictions[i]
        
        i += 1



In [19]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test_final[0],y_pred1[0]))
# print(mape(y_test_final[0],y_pred2[0]))

2673825.0279828818


In [20]:
y_pred = pd.read_csv('exemple_soumission_aleatoire.csv')
x_test = pd.read_csv('x_test.csv')
y_pred['index'] = x_test['index']
x_test = x_test.drop('index',axis=1)
x_test[object_cols] = ordinal_encoder.transform(x_test[object_cols]) 
x_test['date'] = pd.to_datetime(x_train['date'])
# Créer les colonnes pour le jour, le mois et l'année
x_test['jour'] = x_train['date'].dt.day.astype(int)
x_test['mois'] = x_train['date'].dt.month.astype(int)
x_test['année'] = x_train['date'].dt.year.astype(int)
x_test.drop(['date'],axis = 1)


for station, model in models1.items():
    station_data = x_test[x_test['station'] == station]
    X_station = station_data[['job', 'ferie', 'vacances','station','jour','mois','année']]
    pred = model.predict(X_station)
    i=0
    for index in X_station.index:
        # Remplacer la valeur prédite dans y_pred à l'index correspondant
        y_pred.at[index, 'y'] = pred[i]
        i += 1
# Sauvegarder les prédictions dans un fichier CSV
y_pred.to_csv('predictions.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'exemple_soumission_aleatoire.csv'